In [1]:
import sys
sys.path.append("..")
import lib
import numpy as np
import igraph as ig
from tqdm import tqdm

Task Description
- Input:
    - a-z for lowest to highest evelation
    - S for start (evelation a), E for optimal position (evelation z)
- Goal: reach E, saving energy
    - as few steps as possible
    - each step: move !1 square u/d/l/r -> at most one higher than current evelation (but much lower admitted)

In [2]:
print(ord('a'))
print(ord('z'))
print(ord('S'))
print(ord('E'))

97
122
83
69


In [3]:
def are_neighbours(v_a, v_b, verbose = False) -> bool:
    a_idx = np.array([v_a["row"], v_a["col"]])
    b_idx = np.array([v_b["row"], v_b["col"]])
    if np.linalg.norm(a_idx - b_idx, ord = 1) == 1: # Manhattan distance
        return True
    else:
        return False

def admits_edge(vertex, vertex_other, verbose = False):
    # check if there is an edge from vertex to vertex_other
    if vertex_other["height"] - vertex["height"] <= 1: # max. 1 higher or just lower
        return True
    else:
        return False

def get_SE_indices(graph, idx_S, idx_E, verbose = False):
    graph_index_S = None
    graph_index_E = None

    for v in graph.vs:
        if v["row"] == idx_S[0] and v["col"] == idx_S[1]:
            graph_index_S = v.index
        if v["row"] == idx_E[0] and v["col"] == idx_E[1]:
            graph_index_E = v.index
    print(f"Found S, E indices") if verbose else None
    return graph_index_S, graph_index_E

def parse_edges(graph, verbose = False): #-> ig.Graph:
    # go through each vertex
        # check against each vertex if next to it
            # if yes, check if edge condition satisfied
                # if yes, add edge, else None
    edges = []
    print("Starting to parse edges...") if verbose else None
    for vertex in tqdm(graph.vs):
        for vertex_other in graph.vs:
            if are_neighbours(vertex, vertex_other) and admits_edge(vertex, vertex_other):
                edges.append((vertex.index, vertex_other.index))
    graph.add_edges(edges)
    print("Finished parsing edges...") if verbose else None
    return graph

def parse_input(filename : str, verbose : bool = False) -> np.array:
    input = lib.read_file_as_one(filename)
    n_rows = input.count("\n") + 1
    input = input.replace("\n", "")
    n_vertices = len(input)
    n_cols = n_vertices // n_rows
    input = np.array([ord(letter) for letter in input]).reshape((n_rows, -1))
    
    idx_S = np.argwhere(input == ord('S'))[0]
    idx_E = np.argwhere(input == ord('E'))[0]
    input[idx_S[0], idx_S[1]] = ord('a')
    input[idx_E[0], idx_E[1]] = ord('z')
    input = input - ord('a') + 1

    # read into graph class
    vertices_id = [(i,j) for i in range(n_rows) for j in range(n_cols)]
    vertices_row = [i for i in range(n_rows) for j in range(n_cols)]
    vertices_col = [j for i in range(n_rows) for j in range(n_cols)]
    vertices_height = [input[i,j] for i in range(n_rows) for j in range(n_cols)]
    graph = ig.Graph(n = n_rows * n_cols, directed = True)
    graph.vs["id"] = vertices_id
    graph.vs["row"] = vertices_row
    graph.vs["col"] = vertices_col
    graph.vs["height"] = vertices_height

    print(f"Input matrix:\n{input}\n") if verbose else None

    graph = parse_edges(graph, verbose = verbose)
    #print(f"Graph:\n{graph}\n") if verbose else None
    idx_S, idx_E = get_SE_indices(graph, idx_S, idx_E, verbose)

    return graph, idx_S, idx_E

Solution Part A

In [4]:
def compute_partA(filename : str, verbose : bool = False):
    # read file
    graph, idx_S, idx_E = parse_input(filename)
    print(f"Parsed completely, starting to solve...")
    result = graph.distances(source = [idx_S], target = [idx_E])
    # compute solution
    return result[0][0]

def solve_partA():
    result = compute_partA("test_input.txt", False)
    true_result = 31
    assert result == true_result, f"Part A faulty on test file... output = {result}"
    print("Part A works for test file, moving on to whole input...")
    result = compute_partA("input.txt", False)
    print(f"Answer: {result}")
    return

solve_partA()

100%|██████████| 40/40 [00:00<00:00, 3109.08it/s]


Parsed completely, starting to solve...
Part A works for test file, moving on to whole input...


100%|██████████| 7011/7011 [05:31<00:00, 21.13it/s]

Parsed completely, starting to solve...
Answer: 520


Solution Part B

In [7]:
def get_admissible_starts(graph : ig.Graph) -> list:
    admissibles = []
    for v in graph.vs:
        if v["height"] == 1:
            admissibles.append(v.index)
    return admissibles


def compute_partB(filename : str, verbose : bool = False):
    # read file
    graph, idx_S, idx_E = parse_input(filename)
    print(f"Parsed completely, starting to solve...")
    # find admissible set of sources
    admissibles = get_admissible_starts(graph)
    result = graph.distances(source = admissibles, target = [idx_E])
    result = np.array(result).flatten()
    print(f"Distances: {result}")
    return min(result)

def solve_partB():
    result = compute_partB("test_input.txt", True)
    true_result = 29
    assert result == true_result, f"Part B faulty on test file... output = {result}"
    print("Part B works for test file, moving on to whole input...")
    result = compute_partB("input.txt", False)
    print(f"Answer: {result}")
    return

solve_partB()

100%|██████████| 40/40 [00:00<00:00, 2112.15it/s]


Parsed completely, starting to solve...
Distances: [31 30 30 31 30 29]
Part B works for test file, moving on to whole input...


100%|██████████| 7011/7011 [05:34<00:00, 20.96it/s]

Parsed completely, starting to solve...
Distances: [514.  inf  inf ...  inf  inf  inf]
Answer: 508.0
